In [1]:
#This is a part of the solution, for test the application we are developed two training dataset before and
#we are use a dataset that contain an image with anamalies to check the correct functionality of
#functions an libraries developed

In [2]:
import sys
import os
import numpy as np
from numpy import array, asarray, ma, zeros, sum
from matplotlib import pyplot as plt
import cv2
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.cluster.vq import vq, kmeans, whiten, kmeans2
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

os.system('mkdir ./output')
os.system('mkdir ./output/Threats')
os.system('mkdir ./output/NoThreats')
os.system('mkdir ./training')
os.system('mkdir ./results')
os.system('mkdir ./models')
os.system('mkdir ./input')
print("#################################")
print("Directory application structure")
print("#################################")
print(check_output(["ls", "../input/datainput"]).decode("utf8"))
print(check_output(["ls", "../input/modelstraining"]).decode("utf8"))
print(check_output(["ls", "../input/highresolution"]).decode("utf8"))
print(check_output(["ls", "../input/modeldbscan"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

datainput
highresolution
modeldbscan
modelstraining
passenger-screening-algorithm-challenge

#################################
Directory application structure
#################################
0a83698bce92a6824dcc37c1d7fc31f5.csv

model_kmeans.csv

0a83698bce92a6824dcc37c1d7fc31f5_high_resolution.csv

model_dbscan.csv



In [4]:
############################################################################################################
# Name: cluster_analisys
# Autor: Ramiro Bueno Martínez
# Date: 27/11/2017
##########################################################################################################
def cluster_analisys(subset):
	try:
		whitened = whiten(subset)
		codebook,distortion = kmeans(whitened,5)
		return distortion,whitened,codebook
	except Exception as exception:
		print ("cluster_analisys: Excepcion {0}".format(exception))

In [5]:
PATH_PHOTO_FILES = '.'
NAME_FILE_MODEL_KMEANS = '../input/modelstraining/model_kmeans.csv'
NAME_FILE_MODEL_DBSCAN = '../input/modeldbscan/model_dbscan.csv'
DATA_FOR_PREDICTION_FILE = '../input/datainput/0a83698bce92a6824dcc37c1d7fc31f5.csv'
DATA_FOR_PREDICTION_FILE_FILTERED = '../input/highresolution/0a83698bce92a6824dcc37c1d7fc31f5_high_resolution.csv'

In [6]:
############################################################################################################
# Name: creation_dataset_threats
# Autor: Ramiro Bueno Martínez
# Date: 27/11/2017
# Description: The goal of this function is the finding of possible anomalies in the different kind of
# images generated with the body scannes, using clustering techniques based in the analysis of different
# part of the body and different possition imagees
############################################################################################################
def creation_dataset_threats(dataset, FILENAME, highcontrast=False):

		# We have an image with a matrix structure of 660 rows by 512 colums
		# This function send this matrix to a algorithm to obtain different centroids and the distorsion
		# values that we will use to find possible anomalies 
		#Initialization variables
		m_distortion = 0
		col = 0
		row = 0
		nth = 0
		
		
		output_dir = PATH_PHOTO_FILES + '/' + 'output'
		input_dir = PATH_PHOTO_FILES + '/' + 'input'
		
		
		name_of_photo_file = input_dir + '/' + FILENAME
		
		
		print ("Starting clustering Analisys: {0}".format(FILENAME))
		
		#Only they are going to analyse 4 Image Front-Behind-Left-Right
		range_images = [0,4,8,12]
		try:
			
			#for nth in range(16):
			for nth in range_images:
			
				an_img = get_single_image(name_of_photo_file, nth)  				#returns the nth=3 image from the image stack
				
				#Prueba mejorando la resolución de la imagen 
				if highcontrast == True:
					img_rescaled = convert_to_grayscale(an_img)
					an_img = spread_spectrum(img_rescaled)
		
				data_array = np.array(an_img)
				distortion_array[nth,1],whitened,codebook= cluster_analisys(data_array)
				m_distortion = m_distortion + distortion_array[nth,1]
				record = pd.DataFrame([[FILENAME, nth, distortion_array[nth,1],'nothreat']],columns=['Name','Nth','Distortion','Threat'])
				dataset = dataset.append(record,ignore_index=True)
				col = col + 1
				if col % 4 == 0:
					row = row + 1
					col = 0
				print(".")
		
		
		
			#plt.show()
			#print(distortion_array)
			####################################################
			#Esta parte se puede optimizar con las funciones DataFrame.mean()
			media = 0
			media = m_distortion / 4
			#print ("Mean Value of Distortion: {0} \n".format(media))
			############################################################################################
			# At this point we are finding possible anomalies of different distortion measurement with
			# respect to a threshold fixed before
			############################################################################################
			percent = 17  # 17% of the mean value we suppose that its a wrong value 
			############################################################################################
			threshold_thread = media * percent / 100
			for i in range(len(dataset)):
				if abs(media - dataset.loc[i,('Distortion')]) > threshold_thread: 
					dataset.loc[i,('Threat')] = 'threat'
				if abs(media - dataset.loc[i,('Distortion')]) <= threshold_thread:
					dataset.loc[i,('Threat')] = 'nothreat'
					
		except Exception as exception:
			print ("Function [creation_dataset_threats]: Excepcion {0}".format(exception))
			sys.exit(1)
		finally:
			return dataset

In [7]:
if __name__ == "__main__":
	print ("DATA_ANALISIS_TOOL: Initialization of the system...")
	try:
		##############################################################################################
		#APPLICATION OF DIFFERENT KIND OF MODELS TO DETECT POSSIBLE THREATS
		##############################################################################################
		print("Starting model creation based in Support Vector Machine Algorithm ......")
		dataset = pd.read_csv(NAME_FILE_MODEL_KMEANS, sep=',')
		n_samp = len(dataset)
		X, y = make_classification(n_samples=n_samp, n_features=4, random_state=0)
		row = 0
		for index in range(len(dataset)):
			X[row][0]=dataset.loc[index,'Nth']
			X[row][1]=dataset.loc[index,'Distortion']
			X[row][2]=dataset.loc[index,'Difference']
			X[row][3]=dataset.loc[index,'Mean']
			if dataset.loc[index,'Threat'] == 'threat':
				y[index]=np.int32(1)
			else:
				y[index]=np.int32(0)
			
			row = row + 1
		clf = LinearSVC(random_state=0)
		clf.fit(X, y)
		print("Created model ..........")
		print("Starting the process of detection threats .....")
		dataset_new = pd.read_csv(DATA_FOR_PREDICTION_FILE, sep=',')
		data_array = np.array(dataset_new)
		distortion,whitened,codebook = cluster_analisys(data_array)
		mean_value = dataset['Distortion'].mean()
		difference = abs(mean_value - distortion)
		distortion_indicator1 = difference / mean_value * 100
		nth = 3
		result_of_prediction = clf.predict([[nth,distortion,difference,mean_value]]) 	
		name_file = DATA_FOR_PREDICTION_FILE
		name_passenger = str(name_file).split('/')
		name_passenger = str(name_passenger[2]).split('.')
		name_passenger = name_passenger[0]
		print("#############################################################")
		print("Results SECTION 1: Prediction with svm algorithm")
		print("#############################################################")	
		if result_of_prediction == True:
			print("Passenger {0} it could suppose a thread at Body Scanner {1}".format(name_passenger,3))
			threat_result = True
		else:
			print("Passenger {0} it could suppose a thread at Body Scanner {1}".format(name_passenger,3))
		name_file = DATA_FOR_PREDICTION_FILE_FILTERED
		dataset_new = pd.read_csv(DATA_FOR_PREDICTION_FILE_FILTERED, sep=',')
		data_array = np.array(dataset_new)
		distortion,whitened,codebook = cluster_analisys(data_array)
		mean_value = dataset['Distortion'].mean()
		difference = abs(mean_value - distortion)
		distortion_indicator2 = difference / mean_value * 100
		nth = 3
		result_of_prediction = clf.predict([[nth,distortion,difference,mean_value]]) 	
		name_file = DATA_FOR_PREDICTION_FILE_FILTERED
		name_passenger = str(name_file).split('/')
		name_passenger = str(name_passenger[2]).split('.')
		name_passenger = name_passenger[0]
		
		print("###################################################################################")
		print("Results SECTION 2: Prediction with svm algorithm over image with high contrast")
		print("####################################################################################")	
		if result_of_prediction == True:
			print("Passenger {0} it could suppose a thread at Body Scanner {1}".format(name_passenger,3))
			threat_result = True
		else:
			print("Passenger {0} it could suppose a thread at Body Scanner {1}".format(name_passenger,3))
		print("#########################################################################")
		improvement = distortion_indicator2 - distortion_indicator1
		print("Improvement with the filtering image {0} %".format(improvement))
		print("#########################################################################")
 		#print("Results: {0} % of image distortion without".format(distortion_indicator1))
 		#print("Results: {0} % of image distortion with".format(distortion_indicator2))
 		#print("Improvement {0}".foramt(distortion_indicator2-distortion_indicator1))
		sys.exit(0)  #At the demo only the programme do a unique prediction
	except Exception as exception:
		print ("DATA_ANALISIS_TOOL: Excepcion {0}".format(exception))
		sys.exit(1)
	finally:
		print("DATA_ANALISIS_TOOL: That's all, the programe is ended")



DATA_ANALISIS_TOOL: Initialization of the system...
Starting model creation based in Support Vector Machine Algorithm ......
Created model ..........
Starting the process of detection threats .....


/opt/conda/lib/python3.6/site-packages/scipy/cluster/vq.py:141: RuntimeWarning: Some columns have standard deviation zero. The values of these columns will not change.
  RuntimeWarning)


#############################################################
Results SECTION 1: Prediction with svm algorithm
#############################################################
Passenger datainput it could suppose a thread at Body Scanner 3
###################################################################################
Results SECTION 2: Prediction with svm algorithm over image with high contrast
####################################################################################
Passenger highresolution it could suppose a thread at Body Scanner 3
#########################################################################
Improvement with the filtering image -55.25229867048024 %
#########################################################################
DATA_ANALISIS_TOOL: That's all, the programe is ended


SystemExit: 0

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [8]:
#SECTION: 3 APPLICATION DB-SCAN TECHNIQUES 

In [9]:
def dbscan_analysis(data_array):
		print("#############################################################")
		print("Results: SECTION 3 Techniques dbscan application")
		print("#############################################################")	
		whitened = whiten(data_array)
		record = pd.DataFrame({'x_axes': whitened[:, 0],'y_axes':whitened[:, 1]})
		centers = np.array(record)
		X, labels_true = make_blobs(n_samples=750, centers=centers, cluster_std=0.4, random_state=0)
		X = StandardScaler().fit_transform(X)
		db = DBSCAN(eps=0.3, min_samples=10).fit(X)
		core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
		core_samples_mask[db.core_sample_indices_] = True
		labels = db.labels_
		n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
		Name = ''
		nClusters = n_clusters_
		Homegeneity = metrics.homogeneity_score(labels_true, labels)
		Completeness = 	metrics.completeness_score(labels_true, labels)
		Vmeasure = metrics.v_measure_score(labels_true, labels)
		ARIndex = metrics.adjusted_rand_score(labels_true, labels)
		AMInformation = 	metrics.adjusted_mutual_info_score(labels_true, labels)
		SilCoefficient = metrics.silhouette_score(X, labels)
		Threat = 'threat'
		dataset = pd.read_csv(NAME_FILE_MODEL_DBSCAN, sep=',')
		n_samp = len(dataset)
		X, y = make_classification(n_samples=n_samp, n_features=7, random_state=0)
		row = 0
		print('Starting the process of creation of record testing {0}'.format(len(dataset)))
		for index in range(len(dataset)):
			X[row][0]=dataset.loc[index,'nClusters']
			X[row][1]=dataset.loc[index,'Homegeneity']
			X[row][2]=dataset.loc[index,'Completeness']
			X[row][3]=dataset.loc[index,'Vmeasure']
			X[row][4]=dataset.loc[index,'ARIndex']
			X[row][5]=dataset.loc[index,'AMInformation']
			X[row][6]=dataset.loc[index,'SilCoefficient']
			if dataset.loc[index,'Threat'] == 'threat':
				print("Finded a case of threat {0}".format(index))
				y[index]=np.int32(1)
			else:
				print("Finded a case of No threat")
				y[index]=np.int32(0)
			row = row + 1
		print("Starting the process of model scan creation")
		clf = LinearSVC(random_state=0)
		clf.fit(X, y)
		print("Created model dbscan..........")
		
		print('Results: {0}'.format(Homegeneity))